## Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

## Importando arquivos

In [ ]:
df = pd.read_csv('../input/train.csv', parse_dates=[0])
test = pd.read_csv('../input/test.csv', parse_dates=[0])

## Mudando o Nome da Coluna count

In [ ]:
df.rename(columns={'count':'rentals'}, inplace=True)

## Pegando o log da coluna rental, registered e casual:

Somar o +1 serve para evitarmos a operação `log(0)`, que resultar em um valor infinito.

In [ ]:
for col in ['rentals', 'registered', 'casual']:
    df[col] = np.log(df[col] + 1)

## Pré-processamento dos Dados

Juntando os `DataFrames`

In [ ]:
df = df.append(test)

Tratamento de datas

In [ ]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

Criando a coluna `rolling_temp`

In [ ]:
df.set_index('datetime', inplace=True)

In [ ]:
pd.qcut(df['temp'],5).value_counts()

In [ ]:
df.sort_index(inplace=True)

In [ ]:
df['rolling_temp'] = df['temp'].rolling(4, min_periods=1).mean()

In [ ]:
df.reset_index(inplace=True)

Separando os `DataFrames`:

In [ ]:
test = df[df['rentals'].isnull()]

In [ ]:
df = df[~df['rentals'].isnull()]

## Selecionando as Colunas que Iremos Executar o Modelo

In [ ]:
removed_cols = ['rentals', 'casual', 'registered', 'datetime']

In [ ]:
feats = [c for c in df.columns if c not in removed_cols]

## Instanciando o Modelo

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, valid = train_test_split(df, random_state=42)

In [ ]:
train.shape, valid.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(random_state=42)

In [ ]:
rf.fit(train[feats], train['rentals'])

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(rf.predict(valid[feats]), valid['rentals'])**(1/2)

In [ ]:
rf = RandomForestRegressor(random_state=42, n_jobs=-1, max_features=0.8)

In [ ]:
rf.fit(train[feats], train['rentals'])
mean_squared_error(rf.predict(valid[feats]), valid['rentals'])**(1/2)

In [ ]:
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()